## 1. Install Required Dependencies

In [ ]:
# Install required packages
# Run this cell first if packages are not installed
# !uv pip install langchain langchain-google-genai langchain-huggingface chromadb pandas

Using Python 3.12.3 environment at: /mnt/d/source/AI/SmartTraveling/.venv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Audited 5 packages in 890ms


## 2. Import Libraries

In [12]:
import pandas as pd
import unicodedata
import re
import os
from typing import List, Dict, Tuple

# LangChain imports
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate

import warnings
warnings.filterwarnings('ignore')

## 3. Configuration & API Keys

In [ ]:
# Set your Google API key here
# Get your key from: https://makersuite.google.com/app/apikey
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# Configuration
CSV_PATH = '../data/processed/danh_sach_thong_tin_dia_danh_chi_tiet.csv'
CHROMA_DB_PATH = '../data/vector_db/chroma_tourism'
EMBEDDING_MODEL = 'sentence-transformers/multilingual-e5-large-instruct'
TOP_K_RESULTS = 5

## 4. Helper Functions

In [7]:
def slugify(value: str) -> str:
    """
    Convert Vietnamese text with diacritics into a URL-safe slug.
    
    Purpose: Create unique, safe identifiers for each location.
    
    Example:
        "Khu nhà công tử Bạc Liêu" -> "khu_nha_cong_tu_bac_lieu"
        "Thác Khe Vằn" -> "thac_khe_van"
    
    Args:
        value: Vietnamese location name with diacritics
    
    Returns:
        Lowercase slug with underscores (URL-safe)
    """
    # Step 1: Convert Vietnamese 'đ' to 'd' (not handled by NFKD)
    value = str(value).replace("đ", "d").replace("Đ", "D")
    
    # Step 2: Remove Vietnamese diacritics using Unicode normalization
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('utf-8')
    
    # Step 3: Remove special characters, keep only alphanumeric and spaces
    value = re.sub(r'[^\w\s-]', '', value).strip().lower()
    
    # Step 4: Replace spaces and hyphens with underscores
    value = re.sub(r'[\s-]+', '_', value)
    
    return value

## 5. Data Loading & Processing

In [8]:
def load_and_process_data(csv_path: str) -> pd.DataFrame:
    """
    Load tourism location CSV and prepare it for vectorization.
    
    Purpose: 
    - Load raw CSV data
    - Generate loc_id for each location
    - Filter out rows with missing critical fields
    - Set loc_id as index for easy lookup
    
    Args:
        csv_path: Path to the CSV file
    
    Returns:
        Processed DataFrame with loc_id as index
    """
    print("📂 Loading CSV data...")
    df = pd.read_csv(csv_path)
    print(f"   Loaded {len(df)} rows")
    
    # Generate loc_id using slugify
    print("🔑 Generating loc_id for each location...")
    df['loc_id'] = df['TenDiaDanh'].apply(slugify)
    
    # Filter: Keep only rows with TenDiaDanh, DiaChi (NoiDung can be null)
    print("🔍 Filtering rows with missing critical data...")
    df_filtered = df.dropna(subset=['TenDiaDanh', 'DiaChi']).copy()
    
    # Fill NaN in NoiDung with empty string
    df_filtered['NoiDung'] = df_filtered['NoiDung'].fillna('')
    
    print(f"   Kept {len(df_filtered)} rows after filtering")
    
    # Set loc_id as index for fast lookup
    df_filtered = df_filtered.set_index('loc_id')
    
    return df_filtered

In [9]:
# Load the data
tourism_df = load_and_process_data(CSV_PATH)
print(f"\n✅ Data loaded successfully!")
print(f"   Total locations: {len(tourism_df)}")
print(f"\nSample data:")
tourism_df[['TenDiaDanh', 'DiaChi', 'NoiDung']].head(3)

📂 Loading CSV data...
   Loaded 958 rows
🔑 Generating loc_id for each location...
🔍 Filtering rows with missing critical data...
   Kept 858 rows after filtering

✅ Data loaded successfully!
   Total locations: 858

Sample data:


,TenDiaDanh,DiaChi,NoiDung
loc_id,,,
di_tich_lich_su_van_hoa_can_cu_dia_cach_mang_hai_chi_dinh_lang_da,Di tích lịch sử - văn hóa căn cứ địa cách mạng...,"xã Thanh Lâm, Xã Thanh Lâm, Huyện Ba Chẽ, Quản...",Căn cứ địa cách mạng Hải Chi là một địa danh l...
cho_trung_tam_ba_che,Chợ Trung tâm Ba Chẽ,"Thị trấn Ba Chẽ, Thị trấn Ba Chẽ, Huyện Ba Chẽ...",
di_tich_lich_su_mieu_ong_mieu_ba,Di tích lịch sử Miếu Ông – Miếu Bà,"xã Nam Sơn, Xã Nam Sơn, Huyện Ba Chẽ, Quảng Ninh","Miếu Ông và Miếu Bà tọa lạc trên núi Cái Tăn, ..."


## 6. Document Creation

In [10]:
def create_documents(df: pd.DataFrame) -> List[Document]:
    """
    Convert DataFrame rows into LangChain Document objects.
    
    Purpose:
    - Create rich text content for embedding (TenDiaDanh + NoiDung + DiaChi)
    - Store metadata (loc_id, original columns) for retrieval
    - Each Document will be embedded and stored in ChromaDB
    
    Document Structure:
    - page_content: Combined text for semantic search
    - metadata: All original fields for context generation
    
    Args:
        df: Processed DataFrame with loc_id as index
    
    Returns:
        List of LangChain Document objects
    """
    print("📝 Creating documents for vectorization...")
    documents = []
    
    for loc_id, row in df.iterrows():
        # Build rich content: Title + Description + Location
        # This combined text will be embedded for semantic search
        content_parts = [
            f"Tên địa danh: {row['TenDiaDanh']}",
            f"Địa chỉ: {row['DiaChi']}"
        ]
        
        # Add description if available
        if row['NoiDung'] and str(row['NoiDung']).strip():
            content_parts.append(f"Mô tả: {row['NoiDung']}")
        
        page_content = "\n".join(content_parts)
        
        # Store all metadata for later use in recommendations
        metadata = {
            'loc_id': loc_id,
            'TenDiaDanh': row['TenDiaDanh'],
            'DiaChi': row['DiaChi'],
            'NoiDung': row['NoiDung'] if row['NoiDung'] else '',
            'ImageURL': row.get('ImageURL', ''),
            'DichVu': row.get('DichVu', ''),
            'ThongTinLienHe': row.get('ThongTinLienHe', ''),
            'DanhGia': row.get('DanhGia (Google Map)', '')
        }
        
        documents.append(Document(
            page_content=page_content,
            metadata=metadata
        ))
    
    print(f"   Created {len(documents)} documents")
    return documents

In [11]:
# Create documents
documents = create_documents(tourism_df)
print(f"\n✅ Documents created!")
print(f"\nSample document:")
print(f"Content: {documents[0].page_content[:200]}...")
print(f"Metadata keys: {list(documents[0].metadata.keys())}")

📝 Creating documents for vectorization...
   Created 858 documents

✅ Documents created!

Sample document:
Content: Tên địa danh: Di tích lịch sử - văn hóa căn cứ địa cách mạng Hải Chi (Đình Làng Dạ)
Địa chỉ: xã Thanh Lâm, Xã Thanh Lâm, Huyện Ba Chẽ, Quảng Ninh
Mô tả: Căn cứ địa cách mạng Hải Chi là một địa danh lị...
Metadata keys: ['loc_id', 'TenDiaDanh', 'DiaChi', 'NoiDung', 'ImageURL', 'DichVu', 'ThongTinLienHe', 'DanhGia']


## 7. Initialize Embeddings & Vector Store

In [14]:
def initialize_vector_store(documents: List[Document], persist_directory: str) -> Chroma:
    """
    Create embeddings and store in ChromaDB.
    
    Purpose:
    - Initialize HuggingFace embeddings model (runs locally, no API needed)
    - Convert all documents to vectors
    - Store vectors in ChromaDB for fast similarity search
    - Persist to disk for reuse
    
    Why ChromaDB:
    - Persistent storage (no need to re-embed on restart)
    - Fast similarity search
    - Easy integration with LangChain
    
    Args:
        documents: List of LangChain Documents
        persist_directory: Path to store ChromaDB
    
    Returns:
        Initialized Chroma vector store
    """
    print("🤖 Initializing embedding model...")
    print(f"   Model: {EMBEDDING_MODEL}")
    
    # Initialize HuggingFace embeddings (free, runs locally)
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={'device': 'cpu'},  # Use 'cuda' if GPU available
        encode_kwargs={'normalize_embeddings': True}  # Normalize for cosine similarity
    )
    
    print("📦 Creating ChromaDB vector store...")
    print(f"   This may take a few minutes for {len(documents)} documents...")
    
    # Create vector store (embeds all documents)
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name="vietnam_tourism"
    )
    
    print(f"   ✅ Vector store created and persisted to: {persist_directory}")
    return vector_store

In [15]:
# Initialize vector store
vector_store = initialize_vector_store(documents, CHROMA_DB_PATH)
print("\n✅ Vector store ready!")

🤖 Initializing embedding model...
   Model: sentence-transformers/all-MiniLM-L6-v2
📦 Creating ChromaDB vector store...
   This may take a few minutes for 858 documents...
   ✅ Vector store created and persisted to: ../data/vector_db/chroma_tourism

✅ Vector store ready!


## 8. Load Existing Vector Store (Optional)

After the first run, you can load the existing vector store instead of re-creating it:

In [13]:
def load_existing_vector_store(persist_directory: str) -> Chroma:
    """
    Load previously created vector store from disk.
    
    Purpose: Skip re-embedding on subsequent runs (saves time)
    
    Args:
        persist_directory: Path where ChromaDB was persisted
    
    Returns:
        Loaded Chroma vector store
    """
    print("📂 Loading existing vector store...")
    
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vector_store = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name="vietnam_tourism"
    )
    
    print("   ✅ Vector store loaded")
    return vector_store

# Uncomment to load existing store:
# vector_store = load_existing_vector_store(CHROMA_DB_PATH)

## 12. Utility Functions for Analysis

In [28]:
def analyze_results(result: Dict):
    """
    Print detailed analysis of recommendation results.
    """
    print("\n" + "="*60)
    print("📊 DETAILED ANALYSIS")
    print("="*60)
    
    print(f"\n📍 NEW PLACES ({len(result['new_places'])})")
    print("─" * 60)
    for i, doc in enumerate(result['new_places'], 1):
        meta = doc.metadata
        print(f"{i}. {meta['TenDiaDanh']}")
        print(f"   ID: {meta['loc_id']}")
        print(f"   Địa chỉ: {meta['DiaChi']}")
        if meta.get('DanhGia'):
            print(f"   Đánh giá: {meta['DanhGia']}")
        print()
    
    if result['old_places']:
        print(f"\n🔄 VISITED PLACES ({len(result['old_places'])})")
        print("─" * 60)
        for i, doc in enumerate(result['old_places'], 1):
            meta = doc.metadata
            print(f"{i}. {meta['TenDiaDanh']} ({meta['loc_id']})")
    
    print("\n" + "="*60)
    print("📝 FINAL RESPONSE")
    print("═" * 60)
    print(result['final_response'])
    print("═" * 60)

# Use it:
# analyze_results(result)

## 13. Pipeline Summary

### Complete Flow:

```
User Query: "Tôi muốn tìm thác nước đẹp"
           ↓
1. EMBEDDING: Convert query to vector using HuggingFace
           ↓
2. VECTOR SEARCH: Find top-5 similar locations in ChromaDB
           ↓
3. HISTORY FILTER:
   - Check user_visited_ids
   - Split into new_places & old_places
   - Apply allow_revisit logic
           ↓
4. CONTEXT BUILDING: Format location data for LLM
           ↓
5. LLM GENERATION: Gemini creates friendly response
           ↓
Output: Personalized recommendation text
```

### Key Features:

1. **Semantic Search**: Finds locations by meaning, not keywords
2. **Visit History**: Remembers user preferences
3. **Flexible Filtering**: Allow or prevent revisits
4. **Rich Context**: Uses name + description + location
5. **Natural Language**: Gemini generates friendly responses

### Files Created:

- **Vector DB**: `../data/vector_db/chroma_tourism/` (persistent)
- **Documents**: In-memory LangChain Document objects
- **Embeddings**: 384-dimensional vectors (all-MiniLM-L6-v2)

### Next Steps:

1. **Evaluation**: Test with different queries
2. **Fine-tuning**: Adjust prompt template
3. **Integration**: Connect to web/mobile app
4. **Enhancement**: Add filters (price, rating, region)

## Test

In [4]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    # google_api_key=GEMINI_API_KEY,
    )

In [ ]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [ ]:
from langchain.agents import create_agent


tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    """Bạn là một hướng dẫn viên du lịch Việt Nam thân thiện. Nhiệm vụ của bạn:

    1. Luôn đặt câu hỏi khai thác thông tin nếu yêu cầu của người dùng còn mơ hồ hoặc chưa đủ dữ liệu để tư vấn chính xác.

    2. Khi người dùng yêu cầu gợi ý (ví dụ: địa điểm, quán ăn, lịch trình, thông tin lịch sử - văn hóa…), 
    bạn phải tạo danh sách các câu hỏi cần thiết nhằm lấy đủ dữ liệu phục vụ truy xuất từ hệ thống RAG.

    3. Khi câu hỏi của người dùng yêu cầu thông tin thực tế, dữ liệu cụ thể, sự thật lịch sử hoặc chi tiết trong cơ sở dữ liệu, 
    bạn **được phép dùng tool `retriever`**, nhưng chỉ khi:
        - Câu trả lời cần dữ liệu không có trong trí nhớ mô hình.
        - Người dùng yêu cầu thông tin mang tính sự kiện, số liệu, mô tả địa điểm thực tế.
    Nếu không chắc có dữ liệu hoặc độ tin cậy thấp → hãy gọi tool.

    4. Nếu câu hỏi nằm ngoài chuyên môn hoặc không có trong dữ liệu → trả lời: “Tôi không biết”.

    5. Luôn trả lời bằng tiếng Việt thân thiện, tự nhiên và rõ ràng.

    Mục tiêu cuối cùng: đưa ra lời khuyên du lịch chính xác dựa trên dữ liệu, đồng thời chỉ gọi tool khi thật sự cần thiết.
    """
)
agent = create_agent(model, tools, system_prompt=prompt)

In [ ]:
# Test the agent with a sample conversation
test_queries = [
    "Tôi muốn tìm những địa điểm lịch sử cổ kính ở miền Bắc",
    "Có thể bạn gợi ý cho tôi một lịch trình du lịch 3 ngày?",
    "Thác nước nào ở Việt Nam là đẹp nhất?"
]

print("🤖 TESTING AGENT WITH SAMPLE QUERIES\n")
print("="*60)

for i, query in enumerate(test_queries, 1):
    print(f"\n📝 Query {i}: {query}")
    print("-"*60)
    
    try:
        result = agent.invoke({"messages": [("user", query)]})
        
        # Extract the last message from the agent
        last_message = result["messages"][-1]
        response_text = last_message.content
        
        print(f"🤖 Response:\n{response_text}")
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print("-"*60)

print("\n" + "="*60)
print("✅ AGENT TEST COMPLETED")
print("="*60)

🤖 TESTING AGENT WITH SAMPLE QUERIES


📝 Query 1: Tôi muốn tìm những địa điểm lịch sử cổ kính ở miền Bắc
------------------------------------------------------------
🤖 Response:
Chào bạn, rất vui được hỗ trợ bạn. Để tôi có thể gợi ý những địa điểm phù hợp nhất, bạn có thể cho tôi biết thêm một chút về sở thích của mình không? Ví dụ:

*   Bạn thích những di tích gắn liền với triều đại nào (ví dụ: nhà Trần, nhà Lý, nhà Lê...)?
*   Bạn quan tâm đến các loại hình di tích nào (ví dụ: thành quách, đền chùa, lăng tẩm, làng cổ...)?
*   Bạn có dự định đi vào dịp nào không?
*   Bạn thích khám phá những địa điểm mang đậm dấu ấn lịch sử quân sự hay văn hóa, tín ngưỡng?

Càng có nhiều thông tin, tôi càng dễ dàng tìm ra những viên ngọc lịch sử cổ kính dành riêng cho bạn!
------------------------------------------------------------

📝 Query 2: Có thể bạn gợi ý cho tôi một lịch trình du lịch 3 ngày?
------------------------------------------------------------
🤖 Response:
Chào bạn, rất vui được giúp bạn

In [ ]:
!uv pip install -U "psycopg[binary,pool]" langgraph langgraph-checkpoint-postgres

Using Python 3.12.3 environment at: /home/hienlong/projects/Tourism-Chatbot/.venv
Resolved 34 packages in 739ms                                        
⠙ Preparing packages... (0/6)                                                   
⠙ Preparing packages... (0/6)-------------------     0 B/401.43 KiB          
⠙ Preparing packages... (0/6)-------------------     0 B/401.43 KiB          
psycopg-pool         ------------------------------     0 B/39.06 KiB
⠙ Preparing packages... (0/6)-------------------     0 B/401.43 KiB          
psycopg-pool         ------------------------------     0 B/39.06 KiB
⠙ Preparing packages... (0/6)-------------------     0 B/401.43 KiB          
psycopg-pool         ------------------------------     0 B/39.06 KiB
⠙ Preparing packages... (0/6)-------------------     0 B/401.43 KiB          
psycopg-pool         ------------------------------     0 B/39.06 KiB
langgraph-sdk        ------------------------------     0 B/58.86 KiB
⠙ Preparing packages... (0/

In [20]:
import os
import psycopg
import uuid

# Connect to your database
conn = psycopg.connect(f"postgresql://{os.getenv("PSQL_USERNAME")}:{os.getenv("PSQL_PASSWORD")}@{os.getenv("PSQL_HOST")}:5432/{os.getenv("PSQL_DBNAME")}")
conn.autocommit = True  # Important for simple scripts

def ensure_mock_data():
    """Creates a mock user and thread if they don't exist."""
    cur = conn.cursor()
    
    # 1. Generate IDs
    mock_user_id = "00000000-0000-0000-0000-000000000001" # Fixed UUID for easy testing
    mock_thread_id = "00000000-0000-0000-0000-000000000002"
    
    # 2. Insert Mock User (Ignore if exists)
    try:
        cur.execute("""
            INSERT INTO users (id, identifier, metadata)
            VALUES (%s, 'test_user_01', '{}')
            ON CONFLICT (id) DO NOTHING;
        """, (mock_user_id,))
    except Exception as e:
        print(f"User setup error: {e}")

    # 3. Insert Mock Thread (Ignore if exists)
    try:
        cur.execute("""
            INSERT INTO threads (id, name, "userId", "userIdentifier")
            VALUES (%s, 'My Test Chat', %s, 'test_user_01')
            ON CONFLICT (id) DO NOTHING;
        """, (mock_thread_id, mock_user_id))
    except Exception as e:
        print(f"Thread setup error: {e}")
        
    print(f"✅ Setup Complete. Using Thread ID: {mock_thread_id}")
    return mock_thread_id

## 14. Add Memory with PostgreSQL Checkpointer

### Why Add a Checkpointer?

A **checkpointer** enables your agent to:
1. **Remember conversation history** across multiple turns
2. **Persist state** to a database (PostgreSQL)
3. **Resume conversations** even after restart
4. **Track user context** (visited locations, preferences)

### How It Works:

```
User Message → Agent processes → State saved to PostgreSQL
    ↓
Next Message → Agent loads previous state → Continues conversation
```

### Setup Steps:

1. Create PostgreSQL database
2. Configure connection
3. Add checkpointer to agent
4. Test with multi-turn conversation

### Step 1: Database Configuration

First, let's set up the database connection string and verify we can connect.

In [25]:
import os
import psycopg
from psycopg_pool import ConnectionPool

# Database configuration
DB_CONFIG = {
    "host": os.getenv("PSQL_HOST", "localhost"),
    "port": 5432,
    "dbname": os.getenv("PSQL_DBNAME", "tourism_db"),
    "user": os.getenv("PSQL_USERNAME", "tourism"),
    "password": os.getenv("PSQL_PASSWORD")
}

# Build connection string
DB_URI = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

print("📊 Database Configuration:")
print(f"   Host: {DB_CONFIG['host']}")
print(f"   Database: {DB_CONFIG['dbname']}")
print(f"   User: {DB_CONFIG['user']}")

# Test connection
try:
    with psycopg.connect(DB_URI) as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT version();")
            version = cur.fetchone()[0]
            print(f"\n✅ Connected successfully!")
            print(f"   PostgreSQL version: {version[:50]}...")
except Exception as e:
    print(f"\n❌ Connection failed: {e}")

📊 Database Configuration:
   Host: localhost
   Database: tourism_db
   User: tourism

✅ Connected successfully!
   PostgreSQL version: PostgreSQL 16.10 (Ubuntu 16.10-0ubuntu0.24.04.1) o...


### Step 2: Initialize PostgreSQL Checkpointer

The `PostgresSaver` will automatically create the necessary tables (`checkpoints`, `checkpoint_writes`) in your database to store agent state.

In [26]:
from langgraph.checkpoint.postgres import PostgresSaver

# Connection settings for LangGraph
connection_kwargs = {
    "autocommit": True,      # Auto-commit transactions
    "prepare_threshold": 0   # Disable prepared statements (important for compatibility)
}

# Create connection pool
print("🔧 Creating connection pool...")
pool = ConnectionPool(
    conninfo=DB_URI,
    max_size=10,              # Maximum connections
    kwargs=connection_kwargs
)

# Initialize checkpointer
print("💾 Initializing PostgreSQL checkpointer...")
checkpointer = PostgresSaver(pool)

# Setup database schema (creates tables if they don't exist)
print("📋 Setting up database schema...")
checkpointer.setup()

print("\n✅ Checkpointer initialized!")
print("   Tables created: 'checkpoints', 'checkpoint_writes'")

🔧 Creating connection pool...


💾 Initializing PostgreSQL checkpointer...
📋 Setting up database schema...

✅ Checkpointer initialized!
   Tables created: 'checkpoints', 'checkpoint_writes'


### Step 3: Create Agent with Memory

Now we'll recreate the agent using **LangGraph's ReAct pattern** with the checkpointer attached. This enables conversation memory.

In [27]:
from langchain.agents import create_agent
from langchain.tools import tool

# Update the tool to include logging
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve tourism information to help answer a query about Vietnamese destinations."""
    print(f"   🔧 Tool called with query: '{query}'")
    
    retrieved_docs = vector_store.similarity_search(query, k=3)
    
    print(f"   📊 Retrieved {len(retrieved_docs)} documents")
    if retrieved_docs:
        print(f"   📍 Top result: {retrieved_docs[0].metadata.get('TenDiaDanh', 'N/A')}")
    
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

# Define tools
tools = [retrieve_context]

# System prompt for the agent
system_prompt = """Bạn là một hướng dẫn viên du lịch Việt Nam thân thiện, giàu kinh nghiệm.

Nhiệm vụ của bạn:
1. Ưu tiên đưa ra thông tin hữu ích trước, sau đó mới hỏi thêm nếu cần.
2. Sử dụng tool `retrieve_context` khi cần thông tin thực tế về địa danh Việt Nam.
3. Trả lời bằng tiếng Việt tự nhiên, thân thiện và chi tiết.
4. Ghi nhớ ngữ cảnh cuộc hội thoại trước đó.

Khi người dùng hỏi về địa điểm, hãy:
- Gọi tool để lấy thông tin chính xác
- Giới thiệu chi tiết và hấp dẫn
- Gợi ý lý do nên ghé thăm
"""

# Create the agent with checkpointer (enables memory)
print("🤖 Creating agent with memory...")
agent_with_memory = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=checkpointer  # ← This enables memory!
)

print("✅ Agent with memory created!")
print("\n💡 Key feature: Agent now remembers conversation history!")

🤖 Creating agent with memory...
✅ Agent with memory created!

💡 Key feature: Agent now remembers conversation history!


### Step 4: Test Multi-Turn Conversation

Let's test the agent with a conversation that spans multiple turns. The agent should remember context from previous messages.

**Important:** The `thread_id` is like a "conversation ID" - same thread_id = same conversation memory.

In [56]:
import uuid

def chat_with_agent(user_input: str, thread_id: str):
    """
    Send a message to the agent and get response.
    
    Args:
        user_input: User's message
        thread_id: Conversation thread ID (same ID = same conversation)
    
    Returns:
        Agent's response text
    """
    # Configuration with thread_id for memory
    config = {
        "configurable": {
            "thread_id": thread_id  # This links to conversation history
        }
    }
    
    # Prepare input
    inputs = {"messages": [("user", user_input)]}
    
    print(f"\n👤 User: {user_input}")
    print("🤖 Agent thinking...")
    
    response=""
    
    # Invoke agent
    for chunk in agent_with_memory.stream(inputs, config):
        # # Extract response
        # last_message = chunk["messages"][-1]
        # response = last_message.content
        print(chunk)
    
    print(f"🤖 Agent: {response}\n")
    print("─" * 80)
    
    return response

# Generate a unique thread ID for this conversation
conversation_thread_id = '1bb91edb-2276-4d16-8c61-74f8a71fcea9'
print(f"🔑 Conversation Thread ID: {conversation_thread_id}")
print("="*80)

🔑 Conversation Thread ID: 1bb91edb-2276-4d16-8c61-74f8a71fcea9


In [57]:
# Test 1: First message - ask about waterfalls
response1 = chat_with_agent(
    "Tôi muốn tìm những thác nước đẹp ở Việt Nam",
    thread_id=conversation_thread_id
)


👤 User: Tôi muốn tìm những thác nước đẹp ở Việt Nam
🤖 Agent thinking...
{'model': {'messages': [AIMessage(content='Việt Nam mình có rất nhiều thác nước hùng vĩ và tuyệt đẹp đó bạn ơi! Để mình gợi ý cho bạn một vài cái tên nổi bật nhé:\n\n1.  **Thác Bản Giốc (Cao Bằng):** Nằm ở biên giới Việt - Trung, đây là thác nước lớn và đẹp vào bậc nhất Việt Nam. Bạn sẽ choáng ngợp trước vẻ đẹp của dòng thác đổ xuống từ ghềnh đá, tung bọt trắng xóa, bao quanh là cảnh núi non trùng điệp và những cánh đồng xanh mướt. Đặc biệt, vào mùa lúa chín, khung cảnh nơi đây càng thêm phần thơ mộng.\n\n2.  **Thác Datanla (Đà Lạt):** Nếu bạn thích khám phá và trải nghiệm, Datanla là lựa chọn tuyệt vời. Thác không chỉ có vẻ đẹp tự nhiên mà còn có hệ thống máng trượt xuyên rừng độc đáo, mang lại cảm giác phiêu lưu thú vị.\n\n3.  **Thác Pongour (Lâm Đồng):** Được mệnh danh là "Nam thiên đệ nhất thác", Pongour sở hữu vẻ đẹp hoang sơ, kỳ vĩ với hàng ngàn dòng chảy nhỏ len lỏi qua các bậc đá bazan tạo nên một bức tran

In [30]:
# Test 2: Follow-up question (should remember we're talking about waterfalls)
response2 = chat_with_agent(
    "Cái nào gần Đà Nẵng nhất?",
    thread_id=conversation_thread_id  # Same thread = remembers context
)


👤 User: Cái nào gần Đà Nẵng nhất?
🤖 Agent thinking...


🤖 Agent: Dạ, trong các thác nước mình vừa kể thì **Thác Datanla** là gần Đà Nẵng nhất. Thác này nằm ở Đà Lạt, Lâm Đồng. Từ Đà Nẵng, bạn có thể đi máy bay hoặc xe khách đến Đà Lạt, sau đó di chuyển thêm một đoạn ngắn nữa là tới thác Datanla.

Ngoài ra, nếu bạn muốn tìm những thác nước đẹp nằm trong khu vực miền Trung và không quá xa Đà Nẵng, mình có thể tìm hiểu thêm và giới thiệu cho bạn. Bạn có muốn mình tìm hiểu không?

────────────────────────────────────────────────────────────────────────────────


In [31]:
# Test 3: Another follow-up (testing long-term memory)
response3 = chat_with_agent(
    "Tôi đã từng đến nơi đầu tiên bạn gợi ý rồi, nơi nào khác?",
    thread_id=conversation_thread_id  # Still same conversation
)


👤 User: Tôi đã từng đến nơi đầu tiên bạn gợi ý rồi, nơi nào khác?
🤖 Agent thinking...
🤖 Agent: Dạ, bạn đã ghé thăm Thác Bản Giốc rồi ạ. Vậy thì mình sẽ tập trung vào những lựa chọn khác nhé.

Bạn còn nhớ mình đã gợi ý Thác Datanla và Thác Pongour ở Lâm Đồng, cùng Thác Suối Bạc ở Sa Pa.

Nếu bạn muốn tìm một địa điểm khác ngoài những nơi này, bạn có muốn mình tìm hiểu về các thác nước ở khu vực miền Trung hoặc các vùng khác không? Hoặc bạn có tiêu chí nào khác cho chuyến đi của mình không, ví dụ như:

*   Bạn thích sự hoang sơ, hùng vĩ hay vẻ đẹp thơ mộng, trữ tình?
*   Bạn muốn kết hợp tham quan thác nước với các hoạt động khác như trekking, cắm trại?
*   Bạn có dự định đi vào mùa nào trong năm không?

Cho mình biết thêm một chút để mình có thể gợi ý chính xác hơn nhé!

────────────────────────────────────────────────────────────────────────────────


### Step 5: Verify Memory Persistence

Let's verify that the conversation is actually saved in PostgreSQL and can be retrieved.

In [37]:
# Check what's stored in the database
with psycopg.connect(DB_URI) as conn:
    with conn.cursor() as cur:
        # Count checkpoints
        cur.execute("SELECT COUNT(*) FROM checkpoints;")
        checkpoint_count = cur.fetchone()[0]
        
        # Get recent checkpoints
        cur.execute("""
            SELECT thread_id, checkpoint_id 
            FROM checkpoints  
            LIMIT 5;
        """)
        recent_checkpoints = cur.fetchall()
        
        print("📊 Database Status:")
        print(f"   Total checkpoints: {checkpoint_count}")
        print(f"\n📋 Recent checkpoints:")
        for thread_id, checkpoint_id in recent_checkpoints:
            print(f"   - Thread: {thread_id[:20]}... | Checkpoint: {checkpoint_id[:20]}...")
        
        # Check if our conversation is saved
        cur.execute("""
            SELECT COUNT(*) FROM checkpoints 
            WHERE thread_id = %s;
        """, (conversation_thread_id,))
        our_checkpoints = cur.fetchone()[0]
        
        print(f"\n✅ Our conversation has {our_checkpoints} checkpoints saved!")

📊 Database Status:
   Total checkpoints: 33

📋 Recent checkpoints:
   - Thread: 00000000-0000-0000-0... | Checkpoint: 1f0d04c8-d9fb-652c-b...
   - Thread: 00000000-0000-0000-0... | Checkpoint: 1f0d04c8-da01-607f-8...
   - Thread: 00000000-0000-0000-0... | Checkpoint: 1f0d04c9-0d5f-6cf7-8...
   - Thread: 00000000-0000-0000-0... | Checkpoint: 1f0d04c9-4ea4-6ce9-8...
   - Thread: 00000000-0000-0000-0... | Checkpoint: 1f0d04c9-4ea9-60bb-8...

✅ Our conversation has 15 checkpoints saved!


### Step 6: Test with New Thread (No Memory)

Let's create a new conversation with a different thread_id. The agent should NOT remember the previous conversation.

In [33]:
# Start a completely new conversation
new_thread_id = str(uuid.uuid4())
print(f"🔑 New Conversation Thread ID: {new_thread_id}")
print("="*80)

# Ask the same follow-up question from before
# This should fail or ask for clarification since there's no context
response_new = chat_with_agent(
    "Cái nào gần Đà Nẵng nhất?",  # Same question as before
    thread_id=new_thread_id  # Different thread = no memory!
)

print("\n💡 Notice: Agent doesn't know what 'Cái nào' refers to because this is a new conversation!")

🔑 New Conversation Thread ID: 554e6357-25fe-4dd8-938e-6e5ecd7b6108

👤 User: Cái nào gần Đà Nẵng nhất?
🤖 Agent thinking...
🤖 Agent: Chào bạn, để biết địa điểm nào gần Đà Nẵng nhất, bạn có thể cho mình biết bạn đang quan tâm đến loại địa điểm nào không? Ví dụ như bãi biển, danh lam thắng cảnh, khu vui chơi giải trí hay một địa điểm cụ thể nào đó?

Nếu bạn thích khám phá thiên nhiên và lịch sử, **Phố cổ Hội An** là một lựa chọn tuyệt vời. Hội An cách Đà Nẵng khoảng 30km, bạn có thể dễ dàng di chuyển bằng xe máy hoặc ô tô chỉ trong vòng 30-45 phút. Nơi đây nổi tiếng với những ngôi nhà cổ kính, những con đường đèn lồng lung linh về đêm và ẩm thực đặc sắc.

Ngoài ra, nếu bạn muốn tìm kiếm một bãi biển hoang sơ và yên tĩnh hơn, **Biển Lăng Cô** cũng là một điểm đến lý tưởng. Lăng Cô cách Đà Nẵng khoảng 70km về phía Bắc, nổi tiếng với vẻ đẹp tự nhiên hùng vĩ, bãi cát trắng mịn và làn nước trong xanh.

Bạn có thể cho mình biết thêm sở thích của mình để mình có thể gợi ý chính xác hơn nhé!

────

### Step 7: Resume Previous Conversation

Let's go back to the first conversation using the original thread_id. The agent should remember everything!

In [38]:
# Resume the first conversation
print(f"🔑 Resuming Original Thread: {conversation_thread_id}")
print("="*80)

# Continue where we left off
response_continue = chat_with_agent(
    "Cảm ơn! Cho tôi biết thêm về địa điểm đầu tiên nhé",
    thread_id=conversation_thread_id  # Original thread = full memory!
)

print("\n✅ Agent remembered the entire conversation and knows which place is 'địa điểm đầu tiên'!")

🔑 Resuming Original Thread: 1bb91edb-2276-4d16-8c61-74f8a71fcea9

👤 User: Cảm ơn! Cho tôi biết thêm về địa điểm đầu tiên nhé
🤖 Agent thinking...
🤖 Agent: Tuyệt vời! **Thác Bản Giốc** ở tỉnh Cao Bằng là một lựa chọn không thể bỏ qua khi nói đến các thác nước đẹp ở Việt Nam.

Đây là một kỳ quan thiên nhiên hùng vĩ, nằm ngay trên biên giới giữa Việt Nam và Trung Quốc. Điều làm nên sự đặc biệt của Bản Giốc chính là sự rộng lớn và vẻ đẹp ấn tượng của nó:

*   **Quy mô ấn tượng:** Thác Bản Giốc là thác nước lớn thứ tư thế giới nằm trên một đường biên giới quốc gia. Dòng nước từ sông Quây Sơn đổ xuống qua nhiều bậc đá vôi, tạo thành một màn nước trắng xóa, bọt tung trắng xóa, âm thanh vang dội cả một vùng.
*   **Cảnh quan thơ mộng:** Bao quanh thác là khung cảnh núi non trùng điệp, những cánh đồng lúa xanh mướt (đặc biệt đẹp vào mùa lúa chín), và những bản làng dân tộc mộc mạc. Bạn có thể đi thuyền đến gần chân thác để cảm nhận rõ hơn sự hùng vĩ và hơi nước mát lạnh.
*   **Hoạt động trải nghi

## Summary: Memory with Checkpointer

### ✅ What We Accomplished:

1. **PostgreSQL Integration**: Connected agent to database for persistent storage
2. **Conversation Memory**: Agent remembers chat history within same thread
3. **Thread Isolation**: Different threads = separate conversations
4. **Persistence**: Conversations survive restarts (stored in DB)
5. **Tool Integration**: Memory works seamlessly with retrieval tools

### 🔑 Key Concepts:

| Concept | Explanation |
|---------|-------------|
| **thread_id** | Unique ID for each conversation. Same ID = same memory |
| **checkpointer** | Component that saves/loads conversation state |
| **checkpoint** | Snapshot of conversation at a point in time |
| **PostgresSaver** | LangGraph's PostgreSQL storage backend |

### 📊 Database Schema:

```
checkpoints table:
- thread_id: Conversation identifier
- checkpoint_id: Specific state snapshot
- parent_id: Previous checkpoint (for history)
- checkpoint: Serialized state data
- metadata: Additional info
- created_at: Timestamp
```

### 🚀 Benefits:

1. **Natural Conversations**: Users can reference previous messages
2. **Context Awareness**: Agent knows what was discussed
3. **Scalability**: Each user gets their own thread_id
4. **Debugging**: Can inspect saved states in database
5. **Resume Anytime**: Conversations persist across sessions

### 💡 Usage Pattern:

```python
# Same user, same session → use same thread_id
user_thread = f"user_{user_id}"

# First message
chat_with_agent("Tìm bãi biển", thread_id=user_thread)

# Follow-up (agent remembers!)
chat_with_agent("Cái nào đẹp nhất?", thread_id=user_thread)
```

### 🔧 Next Steps:

1. Integrate this into your Chainlit app (`cl_app.py`)
2. Use `cl.user_session.get("id")` as thread_id
3. Add cleanup for old checkpoints
4. Monitor database size over time

## 15. Using Remote Embedding API with gradio_client

Call the embedding API hosted on HuggingFace Spaces using the Python client.


In [11]:
from gradio_client import Client

HF_SPACE_URL = "https://hienlong-tourism-embedding-api.hf.space"

client = Client(HF_SPACE_URL)
client.view_api()

Loaded as API: https://hienlong-tourism-embedding-api.hf.space/ ✔
Client.predict() Usage Info
---------------------------
Named API endpoints: 3

 - predict(text, api_name="/show_embedding") -> embedding_vector_first_10_dims
    Parameters:
     - [Textbox] text: str (required)  
    Returns:
     - [Textbox] embedding_vector_first_10_dims: str 

 - predict(texts, api_name="/show_batch_embeddings") -> results
    Parameters:
     - [Textbox] texts: str (required)  
    Returns:
     - [Textbox] results: str 

 - predict(query, api_name="/similarity_search") -> results
    Parameters:
     - [Textbox] query: str (required)  
    Returns:
     - [Textbox] results: str 

Client.predict() Usage Info
---------------------------
Named API endpoints: 3

 - predict(text, api_name="/show_embedding") -> embedding_vector_first_10_dims
    Parameters:
     - [Textbox] text: str (required)  
    Returns:
     - [Textbox] embedding_vector_first_10_dims: str 

 - predict(texts, api_name="/show_batch_

In [1]:
from gradio_client import Client

# Configure your HF Space URL
HF_SPACE_URL = "https://hienlong-tourism-embedding-api.hf.space"

print("="*70)
print("🔗 CONNECTING TO REMOTE EMBEDDING API")
print("="*70)
print(f"Space URL: {HF_SPACE_URL}\n")

try:
    # Initialize client
    print("📡 Initializing gradio_client...")
    client = Client(HF_SPACE_URL)
    print("✅ Client initialized successfully\n")
    
    # Test 1: Embed single text
    print("📝 Test 1: Embedding single text...")
    test_query = "Thác nước đẹp ở Việt Nam"
    print(f"   Input: '{test_query}'")
    
    embedding = client.predict(
        text=test_query,
        api_name="/embed_text"
    )
    
    print(f"   ✅ Success!")
    print(f"   Embedding dimension: {len(embedding)}")
    print(f"   First 5 values: {embedding[:5]}")
    
    # Test 2: Embed multiple documents
    print("\n📚 Test 2: Embedding multiple documents...")
    test_docs = "Hoi An Ancient Town\nMekong Delta\nHalong Bay"
    print(f"   Input: {len(test_docs.split(chr(10)))} documents")
    
    embeddings = client.predict(
        texts=test_docs,
        api_name="/embed_documents"
    )
    
    print(f"   ✅ Success!")
    print(f"   Generated {len(embeddings)} embeddings")
    print(f"   Each embedding dimension: {len(embeddings[0])}")
    
    # Test 3: Similarity search demo
    print("\n🔍 Test 3: Similarity search demo...")
    search_query = "beaches near Danang"
    print(f"   Query: '{search_query}'")
    
    result = client.predict(
        query=search_query,
        num_results=3,
        api_name="/similarity_search"
    )
    
    print(f"   ✅ Success!")
    print(f"   Result: {result[:100]}...")
    
    print("\n" + "="*70)
    print("✅ ALL TESTS PASSED - API IS WORKING!")
    print("="*70)
    
except Exception as e:
    print(f"\n❌ Error connecting to API: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Check if HF Space is deployed and running")
    print("   2. Verify the Space URL is correct")
    print("   3. Ensure the Space has the embedding API endpoints")
    print("   4. Wait a few minutes if Space is still starting up")

/home/hienlong/projects/Tourism-Chatbot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔗 CONNECTING TO REMOTE EMBEDDING API
Space URL: https://hienlong-tourism-embedding-api.hf.space

📡 Initializing gradio_client...
Loaded as API: https://hienlong-tourism-embedding-api.hf.space/ ✔
✅ Client initialized successfully

📝 Test 1: Embedding single text...
   Input: 'Thác nước đẹp ở Việt Nam'
✅ Client initialized successfully

📝 Test 1: Embedding single text...
   Input: 'Thác nước đẹp ở Việt Nam'
   ✅ Success!
   Embedding dimension: 22631
   First 5 values: [0.01

📚 Test 2: Embedding multiple documents...
   Input: 3 documents
   ✅ Success!
   Embedding dimension: 22631
   First 5 values: [0.01

📚 Test 2: Embedding multiple documents...
   Input: 3 documents
   ✅ Success!
   Generated 67931 embeddings
   Each embedding dimension: 1

🔍 Test 3: Similarity search demo...
   Query: 'beaches near Danang'
   ✅ Success!
   Generated 67931 embeddings
   Each embedding dimension: 1

🔍 Test 3: Similarity search demo...
   Query: 'beaches near Danang'
   ✅ Success!
   Result: Query embe

In [2]:
# Practical Example: Use Remote Embeddings with ChromaDB

print("\n" + "="*70)
print("🔗 INTEGRATION EXAMPLE: Remote Embeddings + ChromaDB")
print("="*70 + "\n")

class RemoteEmbeddingWrapper:
    """Wrapper to make gradio_client compatible with LangChain"""
    
    def __init__(self, space_url: str):
        """Initialize with HF Space URL"""
        from gradio_client import Client
        self.client = Client(space_url)
        print(f"✅ RemoteEmbeddingWrapper initialized: {space_url}")
    
    def embed_query(self, text: str):
        """Embed a single query (LangChain compatible)"""
        try:
            result = self.client.predict(text=text, api_name="/embed_text")
            return result
        except Exception as e:
            print(f"❌ Error embedding query: {e}")
            raise
    
    def embed_documents(self, texts):
        """Embed multiple documents (LangChain compatible)"""
        try:
            # Join texts with newline (API expects newline-separated)
            docs_str = "\n".join(texts)
            results = self.client.predict(texts=docs_str, api_name="/embed_documents")
            return results
        except Exception as e:
            print(f"❌ Error embedding documents: {e}")
            raise


# Create instance
print("🏗️ Creating RemoteEmbeddingWrapper...\n")
try:
    remote_embeddings = RemoteEmbeddingWrapper(HF_SPACE_URL)
    
    # Test it works
    print("\n📝 Testing wrapper with sample query...")
    test_embedding = remote_embeddings.embed_query("Phố cổ Hội An")
    print(f"✅ Query embedding generated: dimension {len(test_embedding)}")
    
    print("\n📚 Testing wrapper with sample documents...")
    test_docs = [
        "Hội An là một thành phố ven biển cũ thuộc tỉnh Quảng Nam cũ (nay là phường Hội An, thành phố Đà Nẵng), Việt Nam.",
        "Đồng bằng sông Cửu Long là bộ phận của châu thổ sông Mê Kông có diện tích 34.699,81 km². Có vị trí nằm liền kề vùng Đông Nam Bộ về phía Đông Bắc, phía Tây Bắc giáp Campuchia, phía Tây Nam là vịnh Thái Lan, phía Đông Nam là Biển Đông.",
        "Vịnh Hạ Long là một vịnh nhỏ thuộc phần bờ tây vịnh Bắc Bộ tại khu vực biển Đông Bắc Việt Nam, bao gồm vùng biển đảo của thành phố Hạ Long thuộc tỉnh Quảng Ninh."
    ]
    test_embeddings = remote_embeddings.embed_documents(test_docs)
    print(f"✅ Document embeddings generated: {len(test_embeddings)} embeddings")
    
    print("\n" + "="*70)
    print("✅ WRAPPER IS READY TO USE WITH CHROMADB!")
    print("="*70)
    
    print("\n💡 Usage with ChromaDB:")
    print("""
    from langchain_chroma import Chroma
    
    # Use remote embeddings with ChromaDB
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=remote_embeddings,  # Your remote embedding wrapper
        persist_directory="./chroma_db"
    )
    
    # Now all similarity searches use remote embeddings!
    results = vector_store.similarity_search(
        "beautiful beaches",
        k=5
    )
    """)
    
except Exception as e:
    print(f"❌ Error: {e}")



🔗 INTEGRATION EXAMPLE: Remote Embeddings + ChromaDB

🏗️ Creating RemoteEmbeddingWrapper...

Loaded as API: https://hienlong-tourism-embedding-api.hf.space/ ✔
✅ RemoteEmbeddingWrapper initialized: https://hienlong-tourism-embedding-api.hf.space

📝 Testing wrapper with sample query...
✅ RemoteEmbeddingWrapper initialized: https://hienlong-tourism-embedding-api.hf.space

📝 Testing wrapper with sample query...
✅ Query embedding generated: dimension 22688

📚 Testing wrapper with sample documents...
✅ Query embedding generated: dimension 22688

📚 Testing wrapper with sample documents...
✅ Document embeddings generated: 68087 embeddings

✅ WRAPPER IS READY TO USE WITH CHROMADB!

💡 Usage with ChromaDB:

    from langchain_chroma import Chroma

    # Use remote embeddings with ChromaDB
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=remote_embeddings,  # Your remote embedding wrapper
        persist_directory="./chroma_db"
    )

    # Now all similarit